In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [8]:
def plot_experiment(experiment_folder, experiment_name):
    con = np.load(experiment_folder + 'con' + experiment_name + '.npz')
    state = np.load(experiment_folder + 'state' + experiment_name + '.npz')
    spikes = np.load(experiment_folder + 'spikes' + experiment_name + '.npz')
    #list_tate['d_E2'], axis=0), label='elems_E2', color='blue')
    
    fig, axs = plt.subplots(4, 1, figsize=(9, 16))
    
    ms = 0.1
    subspikes = 30

    e_1_i_100 = spikes['i'][spikes['i'] < subspikes]
    e_1_t_100 = spikes['t'][spikes['i'] < subspikes]
    
    e_2_i_100 = spikes['i2'][spikes['i2'] < subspikes]
    e_2_t_100 = spikes['t2'][spikes['i2'] < subspikes]

    i_i_100 = spikes['i3'][spikes['i3'] < subspikes]
    i_t_100 = spikes['t3'][spikes['i3'] < subspikes]
    
    
    axs[0].plot(e_1_t_100, e_1_i_100, 'o', label='E1', color='green', markersize=ms, markeredgewidth=0.0)
    axs[0].plot(e_2_t_100, e_2_i_100 + subspikes, 'o', label='E2', color='blue', markersize=ms, markeredgewidth=0.0)
    axs[0].plot(i_t_100, i_i_100 + 2*subspikes, 'o', label='I', color='red', markersize=ms, markeredgewidth=0.0)
    #axs[0].plot(spikes['t'], spikes['i'], ',', label='E1', color='green')
    #axs[0].plot(spikes['t2'], spikes['i2'] + 1000, ',', label='E2', color='blue')
    #axs[0].plot(spikes['t3'], spikes['i3'] + 10000, ',', label='I', color='red')
    
    axs[1].plot(state['t']/1000, np.mean(state['phi_E1'], axis=0), '.', label='phi_E1', color='green')
    axs[1].plot(state['t2']/1000, np.mean(state['phi_E2'], axis=0), '.', label='phi_E2', color='blue')
    
    axs[2].plot(state['t']/1000, np.mean(state['d_E1'], axis=0), '.', label='d_E1', color='green')
    axs[2].plot(state['t2']/1000, np.mean(state['d_E2'], axis=0), '.', label='d_E2', color='blue')
    
    
#    axs[1].plot(state['t'][::5]/1000, np.mean(state['phi_E1'][:,::5], axis=0), '.', label='phi_E1', color='green')
#    axs[1].plot(state['t2'][::5]/1000, np.mean(state['phi_E2'][:,::5], axis=0), '.', label='phi_E2', color='blue')
#    
#    axs[2].plot(state['t'][::5]/1000, np.mean(state['d_E1'][:,::5], axis=0), '.', label='d_E1', color='green')
#    axs[2].plot(state['t2'][::5]/1000, np.mean(state['d_E2'][:,::5], axis=0), '.', label='d_E2', color='blue')

    axs[3].plot(np.array(con['C_E_1_2_t'])/1000, con['C_E_1_2'], '.', label='C_1_2', color='darkgrey')
    axs[3].plot(np.array(con['C_E_2_1_t'])/1000, con['C_E_2_1'], '.', label= 'C_2_1', color='darkgrey')
    axs[3].plot(np.array(con['C_E_1_1_t'])/1000, con['C_E_1_1'], '.', label='C_1_1', color = 'green')
    axs[3].plot(np.array(con['C_E_2_2_t'])/1000, con['C_E_2_2'], '.', label= 'C_2_2', color= 'blue')
    axs[1].set_xlim(0, 300)
    axs[2].set_xlim(0, 300)
    axs[3].set_xlim(0, 300)
    
    return fig, axs

In [4]:
import h5py

# load analytical data from original paper (created with mathematica) 
# and add to existing plot
def plot_analysis(filename, fig, axs):
    f = h5py.File(filename, 'r')
    dset = f['Dataset1']

    phi_E1 = dset[:11501, 2]
    phi_E2 = dset[:11501, 3]
    
    C_E_1_1 = dset[:11501, 4]
    C_E_1_2 = dset[:11501, 5]
    C_E_2_1 = dset[:11501, 6]
    C_E_2_2 = dset[:11501, 7]

    axs[1].plot(np.arange(0, 11501)/10, phi_E1, label='phi_E1', color='green')
    axs[1].plot(np.arange(0, 11501)/10, phi_E2, label='phi_E2', color='blue')

    axs[3].plot(np.arange(0, 11501)/10, C_E_1_1, label='C_1_1', color='green')
    axs[3].plot(np.arange(0, 11501)/10, C_E_2_2, label='C_2_2', color='blue')
    axs[3].plot(np.arange(0, 11501)/10, C_E_1_2, label='C_1_2', color='darkgrey')
    axs[3].plot(np.arange(0, 11501)/10, C_E_2_1, label='C_2_1', color='darkgrey')

In [10]:
folder = 'Experiments/15_03_24_openmp_change_create/'
name = '_order_2500_enable_plasticity_True_tau_ca_2. s_nu_ext_mult_1.1_standalone_False'

fig, axs = plot_experiment(folder, name)
plot_analysis('tracesnaive_tau_ca_2.h5', fig, axs)
#plot_analysis('tracesnaive.h5', fig, axs)
plt.savefig("test.svg")

FileNotFoundError: [Errno 2] No such file or directory: 'Experiments/15_03_24_openmp_change_create/con_order_2500_enable_plasticity_True_tau_ca_2. s_nu_ext_mult_1.05_standalone_False.npz'